In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu102 (from versions: 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==1.9.0+cu102
Note: you may need to restart the kernel to use updated packages.


In [8]:
# 사용자 정의 데이터셋 클래스
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# 데이터 예제 (실제 데이터로 대체 가능)
# 실제로는 DATA_ROOT 경로를 사용하여 데이터를 로드해야 합니다.
# 예를 들어, 이미지 파일을 로드하는 경우 PIL이나 OpenCV를 사용합니다.
data = [1, 2, 3, 4, 5]

# 데이터셋 인스턴스 생성
dataset = MyDataset(data)

# 데이터로더 생성
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# 데이터로더 사용 예시
for batch in data_loader:
    print(batch)

tensor([2, 4])
tensor([1, 5])
tensor([3])


In [9]:
import os
from pathlib import Path
from PIL import Image
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.ndimage import gaussian_filter

import torch
import torch.nn.functional as F
from torch import nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# from google.colab import drive
# drive.mount('/content/drive')

# DATA_ROOT = "/content/drive/MyDrive/Colab Notebooks/Anomaly_detection/Data/MvTec"

In [10]:
# Load wide-ResNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'wide_resnet50_2', pretrained=True)
model.cuda()
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /Users/kim-yuna/.cache/torch/hub/v0.10.0.zip
/Users/kim-yuna/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kim-yuna/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /Users/kim-yuna/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|████████████████████████████████████████| 132M/132M [00:22

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# Set hooks for outputs
def get_activation(name):
    def hook(model, input, output):
        if name not in activation:
            activation[name] = torch.tensor([])
        activation[name] = torch.cat((activation[name], output.detach().cpu()))
    return hook


model.layer2.register_forward_hook(get_activation('layer2'))
model.layer3.register_forward_hook(get_activation('layer3'))

### Normal Dataset

In [ ]:
class BrainDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.data = image_paths

        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = Image.open(self.data[idx])

        if self.transform:
            image = self.transform(image)

        return image

transform = transforms.Compose([
transforms.Resize(256),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# MVTec metal_nut class
train_paths = list(sorted(Path(DATA_ROOT + "/metal_nut/train/good").glob("./*.png")))
test_paths = list(sorted(Path(DATA_ROOT + "/metal_nut/test/color").glob("./*.png")))[:3] + list(sorted(Path(DATA_ROOT + "/metal_nut/test/good").glob("./*.png")))[:2]

train_outputs = OrderedDict([('layer2', []), ('layer3', [])])
test_outputs = OrderedDict([ ('layer2', []), ('layer3', [])])

trainset = BrainDataset(train_paths, transform=transform)
train_dataloader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = BrainDataset(test_paths, transform=transform)
test_dataloader = DataLoader(testset, batch_size=32, shuffle=False)

### Fetures of normal

In [ ]:
activation = {}
with torch.no_grad():
  for x in tqdm(train_dataloader, position =0):
      model(x.cuda())

for layer in ['layer2', 'layer3']:
    train_outputs[layer] = activation[layer]

for layer in ['layer2', 'layer3']:
    print(train_outputs[layer].size())

### Features of abnormal

In [ ]:
activation = {}
with torch.no_grad():
  for x in tqdm(test_dataloader, position =0):
      model(x.cuda())

for layer in ['layer2', 'layer3']:
    test_outputs[layer] = activation[layer]

for layer in ['layer2', 'layer3']:
    print(test_outputs[layer].size())

### Understand how unfold works with examples

In [ ]:
x = torch.from_numpy((100*np.random.random((1, 3, 3, 3))).astype(np.uint8))

plt.imshow(x.squeeze(0).permute(1, 2, 0))
plt.show()
kernel_size = 3
x = F.pad(x, (0, 0, kernel_size//2, kernel_size//2))
print("dim 2 padded  ", x.size())
x = x.unfold(2, kernel_size, 1)
print("dim 2 unfolded", x.size())
x = F.pad(x, (0, 0, kernel_size//2, kernel_size//2, 0, 0))
print("dim 3 padded  ", x.size())
x = x.unfold(3, kernel_size, 1)
print("dim 3 unfolded", x.size())

x = torch.squeeze(x.permute([0, 2, 3, 1, 4, 5]))

plt.imshow(x.permute(2, 0, 3, 1, 4).reshape(3, 3*kernel_size, 3*kernel_size).permute(1, 2, 0))
plt.show()

In [ ]:
x = transforms.ToTensor()(Image.open("IMG_2964.jpeg")).unsqueeze(0)
print("og", x.size()) # [1, 3, 830, 830]

# Assuming kernel is odd, pad kernel//2.
kernel_size = 128
x = F.pad(x, (0, 0, kernel_size//2, kernel_size//2))
print("dim 2 padded  ", x.size())
x = x.unfold(2, kernel_size, step=40)
print("dim 2 unfolded", x.size())
x = F.pad(x, (0, 0, kernel_size//2, kernel_size//2, 0, 0))
print("dim 3 padded  ", x.size())
x = x.unfold(3, kernel_size, step=30)
print("dim 3 unfolded", x.size())

x = torch.squeeze(x.permute([0, 2, 3, 1, 4, 5]))
print("move channel and squeeze", x.size())
print()
print("example patch")
plt.imshow(x[0, 0].permute(1, 2, 0))
plt.show()

print("patched image")
plt.imshow(x.permute(2, 0, 3, 1, 4).reshape(3, x.size()[0]*kernel_size, x.size()[1]*kernel_size).permute(1, 2, 0))
plt.show()

### PatchCore
1. Interpolate and match H, W with largest H, W. <br>
2. Unfold and create patches <br>
3. Adaptive average pool and create locally aware patch features <br>
4. concatine features from different layers and adaptive average pool<br>

In [ ]:
print(test_outputs["layer2"].size())
print(test_outputs["layer3"].size())

In [ ]:
patched_agg_train = []
patched_agg_test = []

print("Here, the steps are")
print()
print("1. H x W -> H_j x W_j 2D interpolation")
print("2. patchify")
print("3. 1D interpolation of C*k*k to pre-defined dim")
print("4. concatinate layers and 1D interpolation to predefined-dim")
print()
print("but in the paper's repo they seem to do 2 and then 1.")
print()

k = 3
m = nn.AdaptiveAvgPool2d(train_outputs['layer2'].size()[2])


for output, patched_agg in [(train_outputs, patched_agg_train), (test_outputs, patched_agg_test)]:
  for layer in ['layer2', 'layer3']:
    # print("---")

    # print("B x C x H x W")
    # print(list(output[layer].size()))
    # print()

    x = F.interpolate(output[layer],
                size=(output['layer2'].size()[-2], output['layer2'].size()[-1]),
                mode="bilinear",
                align_corners=False,
            )

    # print("B x C x H_j x W_j")
    # print(list(x.size()))
    # print()

    # print("**interpolation H x W -> H_j x W_j completed**")
    # print()

    # Pad to make output the same size
    x = F.pad(x, (0, 0, k//2, k//2))

    # print("k//2:", k//2)
    # print()

    # print("B x C x (H_j + 2 * k//2) x W_j")
    # print(list(x.size()))
    # print()

    x = x.unfold(2, k, step=1)

    # print("B x C x H_j x W_j x k")
    # print(list(x.size()))
    # print()

    x = F.pad(x, (0, 0, k//2, k//2, 0, 0))

    # print("B x C x H_j x (W_j + 2 * k//2) x k")
    # print(list(x.size()))
    # print()

    x = x.unfold(3, k, step=1)
    # print("B x C x H_j x W_j x k x k")
    # print(list(x.size()))
    # print()

    # print("**patch creation completed**")
    # print()

    x = x.permute([0, 2, 3, 1, 4, 5])
    # print("B x H_j x W_j x C x k x k")
    # print(list(x.size()))
    # print()

    x = x.reshape(x.size()[0], -1, *x.size()[-3:])
    # print("B x (H_j*W_j) x C x k x k")
    # print(list(x.size()))
    # print()

    x = x.reshape(-1, *x.size()[-3:])
    # print("(B*H_j*W_j) x C x k x k")
    # print(list(x.size()))
    # print()

    x = x.reshape(len(x), 1, -1)
    # print("(B*H_j*W_j) x 1 x (C*k*k)")
    # print(list(x.size()))
    # print()

    x = (F.adaptive_avg_pool1d(x, 1024)).squeeze()
    # print("(B*H_j*W_j) x 1024")
    # print(list(x.size()))
    # print()

    # print("**locally aware patch features created**")
    # print()
    # Aggregate with adaptive average pool

    patched_agg.append(x)

# performing what is done in "Aggregator" class of paper
memory_bank = F.adaptive_avg_pool1d(torch.cat(patched_agg_train, dim=-1), 1024)
test_feats = F.adaptive_avg_pool1d(torch.cat(patched_agg_test, dim=-1), 1024).reshape(testset.__len__(), -1, 1024)

print()
print("memory bank M size:")
print(list(memory_bank.size()))
print()

print("-- steps 1-4 completed --")

### Coreset Sampling (k-center problem) <br>
"choose b center
points such that the largest distance between a data
point and its nearest center is minimized"<br>
ApproximateGreedyCoresetSampler in PatchCore Github

In [ ]:
# based on PatchCore  github
# Compute the distance between the row vectors of X and a single vector y.
def matrix_rows_vs_vec_dist(X, y):

    ### B x 1 x C  @ B x C x 1 (vec inner product) => B x 1 x 1
    X_times_X = X.unsqueeze(1).bmm(X.unsqueeze(2)).reshape(-1, 1)
    y_times_y = y.unsqueeze(1).bmm(y.unsqueeze(2)).reshape(-1, 1).permute(1, 0)
    X_times_y = X.mm(y.T)

    return X_times_X + y_times_y - 2  * X_times_y

In [ ]:
# Greedy approximate algorithm for k-centers problem:
def greedy_k_center(M, k):
  centers = []
  # Choose random center
  cur_center = np.random.randint(len(M))
  centers.append(cur_center)

  # Distance bwtween all points in M and initial center
  dist_to_closest_ctr = matrix_rows_vs_vec_dist(M, M[cur_center].unsqueeze(0))

  for i in tqdm(range(k)):
    # Find point farthest from current ind
    new_center = torch.argmax(torch.squeeze(dist_to_closest_ctr))

    # record new point
    centers.append(new_center)

    # compute distance between all points and y_new
    dist_to_new_center = matrix_rows_vs_vec_dist(M, M[new_center].unsqueeze(0))

    # dist to closest center in the list of centers
    dist_to_closest_ctr = torch.min(torch.cat((dist_to_closest_ctr, dist_to_new_center), dim=-1), dim=-1).values.unsqueeze(1)

  return centers


In [ ]:
# n: reduction factor
n = 0.01

with torch.no_grad():
  # psi: Random Linear Projection

  psi = torch.nn.Linear(memory_bank.size()[-1], 128, bias=False).cuda().eval()
  M_proj = psi(memory_bank.cuda())

  centers = greedy_k_center(M_proj, int(n * len(memory_bank)))

In [ ]:
Mc = memory_bank[centers]
print("-- Memory bank coreset created --")

### Check our greedy k-center algorithm

In [ ]:
N = 1000
points_2d = torch.from_numpy(np.concatenate((np.random.normal(loc=7, scale=4, size=(N, 2)), np.random.normal(loc=-5, scale=2, size=(N, 2)))))

inds = torch.tensor(greedy_k_center(points_2d, 300))
mask = np.full(2*N, True, dtype=bool)
mask[inds] = False

plt.scatter(points_2d[mask][:, 0], points_2d[mask][:, 1])
plt.scatter(points_2d[inds][:, 0], points_2d[inds][:, 1], color="red")
plt.title("k-centers")
plt.show()

In [ ]:
print(Mc.size())
print(test_feats.size())

### Create Score Map

In [ ]:
score_map_list = []
for i in range(len(test_feats)):
  argmin_m = []
  min_m = []
  for y in tqdm(test_feats[i]):
    X = Mc
    y = y.unsqueeze(0)

    min_m.append(torch.min(matrix_rows_vs_vec_dist(X, y)))
    argmin_m.append(torch.argmin(matrix_rows_vs_vec_dist(X, y)))

  m_test_star_ind = np.argmax(min_m)
  m_star_ind = argmin_m[m_test_star_ind]

  s_star = torch.sqrt(torch.sum((test_feats[i][m_test_star_ind] - Mc[m_star_ind])**2))

  # b nearest patch features in Mc of test patch feature m_test_star_ind
  m_test_vec = test_feats[i][m_test_star_ind].unsqueeze(0)

  dist = matrix_rows_vs_vec_dist(Mc, m_test_vec)

  top_b = torch.topk(torch.squeeze(dist), 5).indices
  neighbor_vects = Mc[top_b]

  numerator = torch.exp(s_star)
  denominator = torch.sum(torch.exp(torch.sqrt(torch.sum((neighbor_vects - m_test_vec) ** 2, dim=-1))))

  s = (1-numerator/denominator) * s_star

  score_map = torch.reshape(torch.tensor(min_m), (28, 28))

  score_map_list.append(score_map)


map_interpolated = torch.zeros((len(score_map_list), 224, 224))
for item in range(len(score_map_list)):
    map_interpolated[item] = F.interpolate(score_map_list[item].unsqueeze(0).unsqueeze(0), size=224,mode='bilinear', align_corners=False)

# for item in map_interpolated:
#   plt.imshow(item, vmin=torch.min(map_interpolated), vmax=torch.max(map_interpolated))
#   plt.show()


In [ ]:
def overlay_heatmap_on_image(img, heatmap, ratio_img=0.5):
    img = img.astype(np.float32)

    heatmap = 1 - np.clip(heatmap, 0, 1)
    heatmap = cv2.applyColorMap((heatmap * 255).astype(np.uint8), cv2.COLORMAP_JET)
    heatmap = heatmap.astype(np.float32)

    overlay = (img * ratio_img) + (heatmap * (1 - ratio_img))
    overlay = overlay.astype(np.uint8)
    overlay = np.clip(overlay, 0, 255)
    return overlay


In [ ]:
for item in range(len(map_interpolated)):
    print("---")
    if item < 3:
      print("defect")
    else:
      print("good")
    score_map_ = map_interpolated[item]

    score_map_ = gaussian_filter(score_map_, sigma=4)

    score_map_ = score_map_ - torch.min(map_interpolated).item()
    score_map_ = score_map_ / torch.max(map_interpolated).item()

    # threshold score map

    col_preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ])

    col_anomaly = col_preprocess(Image.open(test_paths[item])).permute(1, 2, 0).numpy() * 255

    plt.imshow(np.clip(col_anomaly.astype(np.uint8), 0, 255))
    plt.show()

    plt.imshow(overlay_heatmap_on_image(col_anomaly, score_map_, 0.5))
    plt.show()

References: <br>
https://tech.anytech.co.jp/entry/2023/03/24/100000 <br>
https://github.com/amazon-science/patchcore-inspection/tree/b64be4734cb8295bfbadccf4f6a036b266181e57/src/patchcore <br>
https://github.com/amazon-science/patchcore-inspection/issues/21#issuecomment-1148390290 <br>
https://www.cs.umd.edu/class/fall2017/cmsc451-0101/Lects/lect08-greedy-k-center.pdf <br>
https://github.com/amazon-science/patchcore-inspection/issues/4#issuecomment-1123763551